## Notebook B: Simulate a steady-state sequence

#### Prerequisites:
- formatted template rawdata.
- a segmentation.
- an XML file describing the segmentation.

#### Goals:
- simulating a static 2D steady state sequence acquisition.

#### Content overview: 
- Setting up the simulation with template data and segmentation.
- Performing the simulation.
- Reconstructing the simulation output.

In [ ]:
from pathlib import Path
import os 
import numpy as np 

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15,7.5]

import sirf.Reg as pReg
import sirf.Gadgetron as pMR
import sirf.DynamicSimulation as pDS

import auxiliary_functions as aux

# this is where we store the properly formatted data
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Input"

fig_path = root_path / "Figures"
fig_path.mkdir(exist_ok=True)


In [ ]:

# set up simulation as we know it from before
fname_xml = fpath_input / "XCAT_TissueParameters_XML.xml"
fname_segmentation = fpath_input / "segmentation.nii"

segmentation = pReg.NiftiImageData3D(str(fname_segmentation))
simulation = pDS.MRDynamicSimulation(segmentation, str(fname_xml))


In [ ]:
# template data
fname_acquisition_template = fpath_input / "acquisition_template.h5"
acquisition_template = pMR.AcquisitionData(str(fname_acquisition_template))

# our template file contains 1500 readouts. We only want to use a subset for this purpose here.
num_acq_subset = np.floor(np.pi/2*256)
subset = np.arange(num_acq_subset)
acquisition_template = acquisition_template.get_subset(subset)

# to activate a golden-angle 2D encoding model we only need to set the trajectory
acquisition_template = pMR.set_goldenangle2D_trajectory(acquisition_template)
simulation.set_template_data(acquisition_template)

Complex Gaussian noise is added to the rawdata after the simulation is performed. The width of the noise distribution is computed based on an SNR parameter and a label of the segmentation for which the SNR is then achieved in image space.

In [ ]:
#
SNR = 10
SNR_label = 13

simulation.set_snr(SNR)
simulation.set_snr_label(SNR_label)

### Coilmaps
The coilmaps used in the simulation can be added flexibly.
The best approach is to first compute them from the rawdata using SIRF and subsequently
- replaced by a simulation of coilmaps that match the data size in the `CoilSensitivityData` object. 
- replaced by a dedicated coilmap which is measured. This will arguably give the best results as they are the most realistic.

It is discouraged to use the computed coil profiles without replacing the data in the `CoilSensitivityData` as anatomy and air which are present in the template acquisitions will be transported into the simulation.

In the auxiliary functions there is an example of how to simulate 2D coilmaps based on using the principal components of Gaussian sensitivity profiles.


In [ ]:
# get gaussian coilmaps
csm = aux.gaussian_2D_coilmaps(acquisition_template)
csm_arr = csm.as_array()

f, ax = plt.subplots(1,4)
for ic in range(4):
    ax[ic].imshow(np.abs(np.squeeze((csm_arr[ic,...]))), vmin=0.0, vmax=0.35)
    ax[ic].axis('off')
    ax[ic].set_title('Receive channel {}'.format(ic+1))

fname_out = fig_path / "fig_b_coilmaps.png"
plt.savefig(str(fname_out), dpi=300)

# 
simulation.set_csm(csm)

In [ ]:

# we set up our transformation to get a 4-chamber view
offset_x_mm = 0
offset_y_mm = 0
offset_z_mm = -14
rotation_angles_deg = [0,0,0]
translation = np.array([offset_x_mm, offset_y_mm, offset_z_mm])
euler_angles_deg = np.array(rotation_angles_deg)

offset_trafo = pReg.AffineTransformation(translation, euler_angles_deg)
simulation.set_offset_trafo(offset_trafo)

In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_b_simulate_motion_static.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)


simulation.write_simulation_results(str(fname_output))

In [ ]:
# reconstruc the data only from without any information on the 
# simulation performed above
simulated_file = pMR.AcquisitionData(str(fname_output))

recon_stat = aux.iterative_reconstruct_data(simulated_file)
recon_stat = pReg.NiftiImageData3D(recon_stat)
recon_stat.write('/media/sf_CCPPETMR/tmp_b_stat.nii')

import matplotlib.pyplot as plt
f = plt.figure()
plt.imshow(np.transpose(np.squeeze(np.abs(recon_stat.as_array()))),cmap='gray')
plt.axis('off')
plt.show()
